In [ ]:
import os
import anndata as ad
import matplotlib as mpl
import numpy as np
import spateo as st

# read raw h5ad adata
data_path = r"D:\BGIpy37_pytorch113\test_data\E16-18_a"
adata = ad.read_h5ad(os.path.join(data_path, "E16-18_a.h5ad"))

Generate whole pcs, surface and volume.

In [ ]:
# generate pcs
pc = st.tl.construct_pc(adata=adata, groupby="anno", colormap="rainbow", key_added="groups")

# generate surface
surf, pc = st.tl.construct_surface(pc=pc, key_added="groups", cs_method="marching_cube", smooth=200, alpha=0.5)
st.pl.three_d_plot(mesh=pc, key="groups", mesh_style="points", point_size=12)
st.pl.three_d_plot(mesh=surf, key="groups", mesh_style="surface")

# generate volume
volume = st.tl.construct_volume(mesh=surf, key_added="groups", volume_smoothness=300)
st.pl.three_d_plot(mesh=volume, key="groups", mesh_style="surface")

# save mesh as a vtk file
st.tl.save_mesh(mesh=pc, filename=os.path.join(data_path, "pcs\E16-18_a_pc.vtk"))
st.tl.save_mesh(mesh=surf, filename=os.path.join(data_path, "surf\E16-18_a_surf.vtk"))
st.tl.save_mesh(mesh=volume, filename=os.path.join(data_path, "volume\E16-18_a_volume.vtk"))

Generate tissues pcs and surface


In [ ]:
tissues = np.asarray(pc.point_data["groups"])
tissues = np.unique(tissues)

tissues_surf = []
for tissue in tissues:
    tissue_pc = st.tl.three_d_pick(mesh=pc, key="groups", picked_groups=tissue)
    block = st.tl.interactive_rectangle_clip(mesh=tissue_pc, key="groups", bg_mesh=None, invert=True)
    trim_tissue_pc = [i for i in block][0]

    color = trim_tissue_pc["groups_rgba"][0]
    hex = mpl.to_hex(c=color, keep_alpha=True)
    tissue_surf, trim_tissue_pc = st.tl.construct_surface(
        pc=trim_tissue_pc, key_added="groups", cs_method="marching_cube", smooth=200, uniform_pc=True, color=hex, alpha=1.0)
    tissue_surf["groups"] = np.asarray([tissue] * tissue_surf.n_cells)

    st.pl.three_d_plot(mesh=trim_tissue_pc, key="groups", mesh_style="points", point_size=12)
    st.pl.three_d_plot(mesh=tissue_surf, key="groups", mesh_style="surface")

    tissues_surf.append(tissue_surf)

Merge all tissues and whole body surface to a complete three d model


In [ ]:
all_surf = tissues_surf
all_surf.append(surf)
mesh = st.tl.collect_mesh(meshes=all_surf)
st.pl.three_d_plot(mesh=mesh, key="groups", mesh_style="surface", opacity=1.0)

# save mesh as a vtk file
st.tl.save_mesh(mesh=mesh, filename=os.path.join(data_path, "mesh.vtm"))